### Import

In [ ]:
import altair as alt

%run implementations.ipynb
%run utilities.ipynb

### Load Data

In [ ]:
sizes = pd.read_csv('npmpackages/npm_no_scope_full_stats_nonzero_downloads.csv', header=None)
sizes.head()

In [ ]:
sizes = pd.read_csv('unsplash/lite/unsplash_lite.csv', header=None)
sizes.head()

### Recall-Precision Analysis

In [ ]:
c_list = [1.01, 1.03, 1.05, 1.07, 1.09]
conf_list = [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.]

size_list = sizes[1].tolist()

size_counts = defaultdict(int)

for size in size_list:
    size_counts[size] += 1
    
uniq_sizes = sorted(list(size_counts.keys()))

In [ ]:
with open('jsvuln/jsvuln.txt') as f:
#with open('nature/nature_urls.txt') as f:
    vuln_packages = set(f.read().splitlines())
    
total_downloads = sizes[2].sum()

size_counts = defaultdict(int)

for row in sizes.itertuples(index=False):
    size_counts[row[1]] += row[2]

for row in sizes.itertuples(index=False):
    if row[0] in vuln_packages:
        print(row[0] + "\t" + str(row[1]) + "\t" + str(row[2]/total_downloads) + "\t" + str(row[2]/size_counts[row[1]]))

In [ ]:
def calc_precision_recall_per_request(size_is_vuln, size_no_vuln, size_counts, c, method, prec_list, recall_list, recall_prec_list):
    for conf in conf_list:
        TP, TN, FP, FN = 0, 0, 0, 0
        for size in size_counts:
            if size_counts[size] > 0.:
                if size_is_vuln[size] / size_counts[size] >= conf:
                    TP += size_is_vuln[size]
                    FP += size_no_vuln[size]
                else:
                    TN += size_no_vuln[size]
                    FN += size_is_vuln[size]
        
        if TP + FP > 0:
            prec = TP / (TP + FP)
            prec_list[(c, method, conf)] = prec
        
        if TP + FN > 0:
            recall = TP / (TP + FN)        
            recall_list[(c, method, conf)] = recall
            
        if TP + FP > 0 and TP + FN > 0:
            recall_prec_list[(c, method, conf)] = (recall, prec)

In [ ]:
def calc_precision_recall(data, c, method, prec_list, recall_list, recall_prec_list):
    size_is_vuln = defaultdict(int)
    size_no_vuln = defaultdict(int)
    size_counts = defaultdict(int)

    for row in data.itertuples(index=False):
        if row[0] in vuln_packages:
            size_is_vuln[row[1]] += row[2]
        else:
            size_no_vuln[row[1]] += row[2]
        size_counts[row[1]] += row[2]

    for conf in conf_list:
        TP, TN, FP, FN = 0, 0, 0, 0
        for size in size_counts:
            if size_counts[size] > 0.:
                if size_is_vuln[size] / size_counts[size] >= conf:
                    TP += size_is_vuln[size]
                    FP += size_no_vuln[size]
                else:
                    TN += size_no_vuln[size]
                    FN += size_is_vuln[size]
        
        if TP + FP > 0:
            prec = TP / (TP + FP)
            prec_list[(c, method, conf)] = prec
        
        if TP + FN > 0:
            recall = TP / (TP + FN)        
            recall_list[(c, method, conf)] = recall
            
        if TP + FP > 0 and TP + FN > 0:
            recall_prec_list[(c, method, conf)] = (recall, prec)

In [ ]:
prec_list = {}
recall_list = {}
recall_prec_list = {}

In [ ]:
for c in c_list:
    size_is_vuln, size_no_vuln, size_counts = BlahutArimoto_return_vuln(sizes, c, vuln_packages, max_itr=10000000, eps=5e-3)
    
    calc_precision_recall_per_request(size_is_vuln, size_no_vuln, size_counts, c, "MI_per_request", prec_list, recall_list, recall_prec_list)
    print(c)

In [ ]:
for c in c_list:
    size_is_vuln, size_no_vuln, size_counts = p_alpaca_no_metrics_return_vuln(sizes, c, vuln_packages)

    calc_precision_recall_per_request(size_is_vuln, size_no_vuln, size_counts, c, "P-ALPaCA", prec_list, recall_list, recall_prec_list)
    print(c)

In [ ]:
for c in c_list:
    partition, dyn_solution = opt_partition_MI(sizes, c)

    size_map = {}    
    current_block = 0
    current_ceiling = partition[current_block][1]

    for size in uniq_sizes:
        if size > current_ceiling:
            current_block += 1
            current_ceiling = partition[current_block][1]
        size_map[size] = current_ceiling

    dyn_list = []

    for row in sizes.itertuples(index=False):
        dyn_size = size_map[row[1]]
        dyn_list.append((row[0], dyn_size, row[2]))
    
    dyn_sizes = pd.DataFrame(dyn_list)

    calc_precision_recall(dyn_sizes, c, "MI_per_object", prec_list, recall_list, recall_prec_list)
    print(c)

In [ ]:
for c in c_list:
    size_map = pureMaxL(sizes, c)    
    current_block = 0
    current_ceiling = partition[current_block][1]

    dyn_list = []

    for row in sizes.itertuples(index=False):
        dyn_size = size_map[row[1]]
        dyn_list.append((row[0], dyn_size, row[2]))
    
    dyn_sizes = pd.DataFrame(dyn_list)

    calc_precision_recall(dyn_sizes, c, "Pure_MaxL", prec_list, recall_list, recall_prec_list)
    print(c)

In [ ]:
padme_list = []

for row in sizes.itertuples(index=False):
    padme_size = getPadme(row[1])
    padme_list.append((row[0], padme_size, row[2]))
    
padme_sizes = pd.DataFrame(padme_list)

calc_precision_recall(padme_sizes, 1.03, "Padme", prec_list, recall_list, recall_prec_list)

In [ ]:
for c in c_list:
    min_size = sizes[1].min()
    bin_size = int(c*min_size) - min_size
    
    d_alpaca_list = []

    for row in sizes.itertuples(index=False):
        d_alpaca_size = getDALPaCA(row[1], bin_size)
        d_alpaca_list.append((row[0], d_alpaca_size, row[2]))
    
    d_alpaca_sizes = pd.DataFrame(d_alpaca_list)

    calc_precision_recall(d_alpaca_sizes, c, "D-ALPaCA", prec_list, recall_list, recall_prec_list)
    print(c)

In [ ]:
new_prec_list = [0] * len(prec_list)
new_recall_list = [0] * len(recall_list)
new_recall_prec_list = [0] * len(recall_prec_list)

i = 0
for key, value in prec_list.items():
    new_prec_list[i] = (key[0], key[1], key[2], value)
    i += 1
    
i = 0
for key, value in recall_list.items():
    new_recall_list[i] = (key[0], key[1], key[2], value)
    i += 1
    
i = 0
for key, value in recall_prec_list.items():
    new_recall_prec_list[i] = (key[0], key[1], key[2], value[0], value[1])
    i += 1

In [ ]:
df_prec = pd.DataFrame(new_prec_list, columns =['c', 'Method', 'Confidence', 'Precision'])
df_recall = pd.DataFrame(new_recall_list, columns =['c', 'Method', 'Confidence', 'Recall'])
df_recall_prec = pd.DataFrame(new_recall_prec_list, columns =['c', 'Method', 'Confidence', 'Recall', 'Precision'])

In [ ]:
df_recall_prec_temp = df_recall_prec.loc[df_recall_prec['c'] == 1.09, ['Method', 'Recall', 'Precision']]

alt.Chart(df_recall_prec_temp).mark_line().encode(
    x='Recall',
    y='Precision',
    color='Method',
    strokeDash='Method',
)

In [ ]:
df_recall_prec.to_csv('evaluation/recall-precision-unsplash.csv')

__Print__ $\LaTeX{}$

In [ ]:
df_recall_prec = pd.read_csv('evaluation/recall-precision-unsplash.csv')
df_recall_prec.head()

In [ ]:
methods = ["MI_per_request", "MI_per_object", "P-ALPaCA", "Pure_MaxL", "Padme", "D-ALPaCA"]
legend = {
          "MI_per_request": "[line width=1pt, densely dotted, curvecolor]", 
          "MI_per_object": "[line width=1pt, dash pattern=on 1pt off 3pt on 3pt off 3pt, curvecolor]", 
          "P-ALPaCA": "[line width=1pt, densely dotted, black]", 
          "Pure_MaxL": "[line width=1pt, solid, curvecolor]", 
          "Padme": "[line width=1pt, solid, black]", 
          "D-ALPaCA": "[line width=1pt, dash pattern=on 1pt off 3pt on 3pt off 3pt, black]"
         }

In [ ]:
df_recall_prec_temp = df_recall_prec.loc[df_recall_prec['c'] == 1.09, ['Method', 'Recall', 'Precision']]

for method in methods:
    df_method_temp = df_recall_prec_temp.loc[df_recall_prec_temp['Method'] == method, ['Recall', 'Precision']]
    print("\\addplot " + legend[method])
    print("table {%")
    for row in df_method_temp.itertuples():
        print(str(row.Recall) + ' ' + str(row.Precision))        
    print("};")